<a href="https://colab.research.google.com/github/basanab/mu_e_scatt/blob/main/read_ROOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install uproot    # new

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import uproot
import math as mt

In [ ]:
file = uproot.open("/content/drive/MyDrive/ColabNotebooks/beryllium/scat_Be_500GeV_10Cr.root")
file.keys()
file.classnames()
branch=file["mu_e_scatt"]
branch.keys()

In [ ]:
df=branch.arrays(['event_id', 'mu_x', 'mu_y', 'mu_z', 'e_x', 'e_y', 'e_z', 'mu_E','electron_E','mu_px',
 'mu_py',
 'mu_pz',
 'e_px',
 'e_py',
 'e_pz',
 'scat_type'], library="pd")
# df

In [ ]:
df.to_csv('Be_500geV_10Cr.csv', header=True, index=False, sep='\t', mode='a')

In [ ]:
#to separate out the elastic scattering type
df_sort1=df_Be_500_10Cr[df_Be_500_10Cr['scat_type']==1]
#reset index without adding new column "index" , as drop=true
df_sort1.reset_index(level=None, drop=True, inplace=True)
print(df_sort1)
#This will give the number of rows in all columns where the value = 1
(df_sort1==1).sum()

In [ ]:
me= 0.511 * 10**(-3) #in units of GeV/c^2
mm= 207* me
pi= np.sqrt(500**2- mm**2)
pi # in units of GeV
deter=[]
for i in range(len(df_sort1)):
    matx=np.array([[0,0,-pi*1000],[df_sort1["mu_px"][i],df_sort1["mu_py"][i],df_sort1["mu_pz"][i]],[df_sort1["e_px"][i],df_sort1["e_py"][i],df_sort1["e_pz"][i]]])
    deter.append(np.linalg.det(matx))
#momentum units are taken in MeV by Geant4
print(deter)


In [ ]:
df_sort1["determinant"]=deter
df_sort1

In [ ]:
# to check the coplanarity factor that is determinant = 0
sort2=df_sort1[df_sort1["determinant"]==0.0]
#reset index without adding new column "index" , as drop=true
sort2.reset_index(level=None, drop=True, inplace=True)
sort2

In [ ]:
# To subtract the thickness of target (0.005 mm)
sort2['e_z'] = sort2['e_z']+0.005
sort2['mu_z'] = sort2['mu_z']+0.005

In [ ]:
# to find the scayttering angles from the sorted datasheet
tm =np.pi-np.arccos( sort2.iloc[:,3]/np.sqrt( sort2.iloc[:,1]**2+  sort2.iloc[:,2]**2+  sort2.iloc[:,3]**2))
te = np.pi -np.arccos( sort2.iloc[:,6]/np.sqrt( sort2.iloc[:,4]**2+  sort2.iloc[:,5]**2+  sort2.iloc[:,6]**2))

In [ ]:
resul_ang= pd.DataFrame({"tm":tm,"te":te},columns=['tm','te'])#to make resultant angles as separate dataframes
# to add the angles column with the sorted dataframe
ang_sort = pd.concat([sort2,resul_ang], axis=1)
ang_sort